# A simple RAG application using open-source models

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
# MODEL = "mixtral:8x7b"
# MODEL = "llama2"


In [2]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

AIMessage(content='Why did the scarecrow win an award?\nBecause he was outstanding in his field!', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e366f387-c9ea-4612-a40c-f6135454ee86-0')

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired!"

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'My name is Santiago.'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Oxford Law.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Back to  home page... Herts Insight\nGeodemographic Pro\x00le for Hertfordshire\nThis pro\x00le looks at geodemographic data in Hertfordshire.\xa0\nContents\n\xa0Geodemographic Classi\x00cationView this Pro\x00le for...\nHertfordshire Districts\nHertfordshire Wards\nHertfordshire Electoral Divisions\nContains OS data © Crown Copyright and d… Powered by Esri5/12/24, 3:48 PM Geodemographic Profile - County | Hertfordshire | Report Builder for ArcGIS\nhttps://www.reports.esriuk.com/view-report/b231b0da9c7a464fb84317621758332c/E10000015?clear=true 1/5', metadata={'source': 'Demographic.pdf', 'page': 0}),
 Document(page_content='Geodemographic Classi\x00cation\nGeodemographic segmentation is a classi\x00cation technique which groups people and households according to their likely\ncharacteristics.\xa0 In short it is a tool which combines where people live, their demographics, behaviour and lifestyle information\nto give us an understanding of the types of people livi

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [8]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

[Document(page_content='Geodemographic Classi\x00cation\nGeodemographic segmentation is a classi\x00cation technique which groups people and households according to their likely\ncharacteristics.\xa0 In short it is a tool which combines where people live, their demographics, behaviour and lifestyle information\nto give us an understanding of the types of people living in a particular area. The Acorn segmentation is used here.\nThe present version of Acorn draws on a wide variety of data sources, both commercial and public sector open data and ad -\nministrative data. This includes land registry, commercial data on ages of residents, bene\x00ts data, population density, care\nhomes, social housing and rental properties. It uses a methodology that recognises that census data is no longer the most up\nto date source for neighbourhoods.\nThe Acorn segmentation is provided by CACI and classi\x00es households into one of six categories.\nThe A\x00uent Achievers category is the majority in ho

In [9]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [12]:
questions = [
    "How many geodemographic classifications are there?",
    "What are the percentages per each classification?",
    
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: How many geodemographic classifications are there?
Answer: There are six geodemographic classifications.

Question: What are the percentages per each classification?
Answer: I don't know.



In [11]:
chain.batch([{"question": q} for q in questions])

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-ada-002 in organization org-IOecdRYGmVvHe996VIrIaHac on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
for s in chain.stream({"question": "What is the purpose of the course?"}):
    print(s, end="", flush=True)

In [ ]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = WebBaseLoader("https://www.ml.school")
# docs = loader.load()
# documents = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200
# ).split_documents(docs)

# documents